In [ ]:
import heapq
from collections import defaultdict
import bitarray
from google.colab import files
import os


# Paso 1: Leer archivo de texto de entrada
def leer_archivo():
    uploaded_files = files.upload()  # Cargar archivo desde el sistema local
    nombre_archivo = next(iter(uploaded_files))  # Obtener el nombre del archivo cargado
    texto_original = uploaded_files[nombre_archivo].decode('utf-8')  # Decodificar contenido del archivo a una cadena de texto

    # Obtener el peso del archivo original
    peso_original = os.path.getsize(nombre_archivo)
    print("\033[1;34m--------------------------------------------------------------\033[0m")
    print("\033[1;34m Peso del archivo original: \033[0m", peso_original, "\033[1;34m bytes \033[0m"+"\n")
    return texto_original

# Paso 2: Calcular frecuencias de caracteres
def calcular_frecuencias(texto):
    frecuencias = defaultdict(int)
    for caracter in texto:
        frecuencias[caracter] += 1  # Contar la frecuencia de cada carácter en el texto
    return frecuencias

# Paso 3: Construir árbol de Huffman
def construir_arbol(frecuencias):
    cola_prioridad = [[frecuencia, [caracter, ""]] for caracter, frecuencia in frecuencias.items()]
    heapq.heapify(cola_prioridad)  # Convertir la lista en una cola de prioridad
    while len(cola_prioridad) > 1:
        lo = heapq.heappop(cola_prioridad)  # Sacar el elemento de menor prioridad de la cola
        hi = heapq.heappop(cola_prioridad)  # Sacar el siguiente elemento de menor prioridad
        for par in lo[1:]:
            par[1] = '0' + par[1]  # Asignar '0' al código del carácter en el subárbol izquierdo
        for par in hi[1:]:
            par[1] = '1' + par[1]  # Asignar '1' al código del carácter en el subárbol derecho
        heapq.heappush(cola_prioridad, [lo[0] + hi[0]] + lo[1:] + hi[1:])  # Insertar el nuevo nodo en la cola con su frecuencia actualizada
    return cola_prioridad[0]  # Devolver el nodo raíz del árbol de Huffman

# Paso 4: Generar códigos Huffman
def generar_codigos(arbol):
    codigos = {}
    for caracter, codigo in arbol[1:]:
        codigos[caracter] = codigo  # Asignar los códigos Huffman a cada carácter del árbol
    return codigos

# Paso 5: Codificar texto
def codificar_texto(texto, codigos):
    texto_codificado = bitarray.bitarray()
    for caracter in texto:
        texto_codificado.extend(bitarray.bitarray(codigos[caracter]))  # Agregar el código Huffman del carácter al texto codificado

    # Obtener el peso del texto codificado
    peso_codificado = len(texto_codificado) * 0.125
    print("\033[1;34m Peso del archivo codificado: \033[0m", peso_codificado, "\033[1;34m bytes \033[0m"+"\n")

    return texto_codificado

# Paso 6: Calcular tasa de compresión
def calcular_tasa_compresion(texto_original, texto_codificado):
    tasa_compresion = (1 - (len(texto_codificado) / (8 * len(texto_original)))) * 100  # Calcular la tasa de compresión en porcentaje
    return tasa_compresion

# Paso 7: Decodificar texto
def decodificar_texto(texto_codificado, codigos):
    codigo_inverso = {codigo: caracter for caracter, codigo in codigos.items()}
    texto_decodificado = ""
    codigo_actual = ""
    for bit in texto_codificado:
        codigo_actual += str(bit)  # Agregar el bit actual al código en construcción
        if codigo_actual in codigo_inverso:
            caracter = codigo_inverso[codigo_actual]  # Obtener el carácter correspondiente al código actual
            texto_decodificado += caracter  # Agregar el carácter al texto decodificado
            codigo_actual = ""  # Reiniciar el código actual
    return texto_decodificado

# Función auxiliar para guardar archivo de texto
def guardar_archivo(nombre_archivo, contenido):
    with open(nombre_archivo, 'w') as archivo:
        archivo.write(contenido)  # Escribir el contenido en el archivo

# Función auxiliar para mostrar resultado
def mostrar_resultado(tasa_compresion, texto_decodificado):
    print("\033[1;34m--------------------------------------------------------------\033[0m")
    print("\033[1;34m El texto se ha codificado correctamente. \033[0m"+"\n")
    print("Tasa de compresión: \033[1;34m {:.2f}%".format(tasa_compresion))  # Mostrar la tasa de compresión en la terminal
    print("\033[1;34m--------------------------------------------------------------\033[0m")

# Función auxiliar para mostrar contenido del archivo codificado en un párrafo
def mostrar_contenido_codificado(codigos):
    print("\033[1;34m \nContenido del archivo codificado: \033[0m")
    for caracter, codigo in codigos.items():
        print(f"{caracter}: {codigo}")

# Paso 0: Ejecutar el sistema de compresión
texto_original = leer_archivo()  # Leer el archivo de texto de entrada
frecuencias = calcular_frecuencias(texto_original)  # Calcular las frecuencias de los caracteres en el texto
arbol = construir_arbol(frecuencias)  # Construir el árbol de Huffman utilizando las frecuencias
codigos = generar_codigos(arbol)  # Generar los códigos Huffman para cada carácter
texto_codificado = codificar_texto(texto_original, codigos)  # Codificar el texto original utilizando los códigos Huffman
tasa_compresion = calcular_tasa_compresion(texto_original, texto_codificado)  # Calcular la tasa de compresión
texto_decodificado = decodificar_texto(texto_codificado, codigos)  # Decodificar el texto codificado utilizando los códigos Huffman

# Guardar archivo de texto codificado
nombre_archivo_codificado = "texto_codificado.txt"
with open(nombre_archivo_codificado, 'w') as archivo_codificado:
    archivo_codificado.write(texto_codificado.to01())  # Guardar el contenido codificado en el archivo



# Mostrar resultado
mostrar_resultado(tasa_compresion, texto_decodificado)
mostrar_contenido_codificado(codigos)

# Mostrar contenido del archivo decodificado
print("\033[1;34m--------------------------------------------------------------\033[0m")
print("\033[1;34m Contenido del archivo decodificado: \033[0m"+"\n")
print(texto_decodificado)  # Mostrar el contenido decodificado en la terminal
print("\033[1;34m--------------------------------------------------------------\033[0m")